## Multiclass Logistic Regression

In [74]:
import numpy as np

class LogisticRegression():
    
    def __init__(self):
        
        self.W = None
        
        
    @staticmethod
    def softmax(arr):
        """
        Rowwise computation of softmax.
        Input:
         - arr: an array of dimension n x k (in this context X @ W.T)
        Output:
         - an array of the same dimension as input
        """
        a = arr.max(axis=1)
        ex_arr = np.exp(arr - a[:,None])
        return ex_arr / ex_arr.sum(axis=1)[:,None]
    
    @staticmethod
    def log_sum_exp(arr):
        """
        Rowwise computation of log-sum-exp.
        Input: 
         - arr: an array of dimension n x k (in this context X @ W.T)
        Output:
         - an array of dimension n x 1
        """
        a = arr.max(axis=1)
        return a + np.log(np.sum(np.exp(arr - a[:,None]), axis=1))
    
    @staticmethod
    def one_hot(vec):
        """
        One hot encoder for response:
        Input:
         - vec: an n x 1 (response) vector 
        Output:
         - an n x k one hot encoding of the input
        """
        ## one hot encoding from: https://stackoverflow.com/questions/29831489/numpy-1-hot-array
        one_hot = np.zeros((len(vec), len(np.unique(vec))))
        one_hot[np.arange(len(vec)), vec] = 1
        return one_hot
    
    @staticmethod
    def gradient_descent(f, grad_f, w0, alpha=0.001, eps=0.005):
        """
        Gradient Descent for fitting:
        Input: 
         - f:      the function to be minimized (loss in this context)
         - grad_f: the gradient of the function to be minimized
         - w0:     an initial guess
         - alpha:  the learning rate (constant at this point)
         - eps:    the tolerance on the magnitude of the gradient
        Outputs:
         - a k x d dimensional array of weights w that minimizes f
        """
        w = w0
        g = grad_f(w)
        while np.linalg.norm(g) > eps:  
            g = grad_f(w)
            w = w - alpha * grad_f(w)
            print(np.linalg.norm(g))
        return w
    
    @staticmethod
    def loss(W, X, y):
        """
        Loss for Logistic Regression 
        Inputs:
         - W: a k x d weight matrix where rows are the d weights for kth class
         - X: the n x d data matrix (n observations, d features)
         - y: the target classes
        Outputs:
         - a number that should mean something to someone
        """
        one_hot = LogisticRegression.one_hot(y)

        W = W.reshape((len(np.unique(y)), X.shape[1]))
        xw = X @ W.T 
        lse = LogisticRegression.log_sum_exp(xw)
        return -np.sum(one_hot * (xw - lse[:,None])) 
    
    @staticmethod
    def grad_loss(W, X, y):
        """
        Inputs:
         - W: k x d array of parameters
         - X: n x d design matrix
         - y: n x 1 target vector
        Output:
         - a k x d array of partial derivatives
        """
        one_hot = LogisticRegression.one_hot(y)

        W = W.reshape((len(np.unique(y)), X.shape[1]))

        g = X.T @ (one_hot - LogisticRegression.softmax(X @ W.T)) 
        return -g.T #-g.flatten(order='F')

    def fit(self, X, y):
        """
        Fit method:
        Input:
         - X: an n x d design matrix
         - y: a n x 1 target vector
        Ouput:
         - a k x d array of optimal weights
        """
        f = lambda w: LogisticRegression.loss(w, X, y)
        grad_f = lambda w: LogisticRegression.grad_loss(w, X, y)
        w0 = np.random.rand(len(np.unique(y))*X.shape[1])
        w0 = w0.reshape((len(np.unique(y)), X.shape[1]))
        self.W = LogisticRegression.gradient_descent(f, grad_f, w0)
        return None
        
        
    def predict(self, X):
        """
        Multiclass soft prediction method:
        Input:
         - X: an n x d design matrix
        Output:
         - a n x k array of probabilistic predictions
           each observation gets k predicted probabilities 
         - note: if you want hard predictions, use out.argmax(axis=1)
           where out is what this method returns
        """
        return LogisticRegression.softmax(X @ self.W.T)
    


In [59]:
#from sklearn.linear_model import LogisticRegression
import sklearn.datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [60]:
# load some dataset to check how it works
digits = sklearn.datasets.load_digits()
X, Xvalidate, y, yvalidate = train_test_split(digits.data, digits.target)


In [75]:
clf = LogisticRegression()
clf.fit(X,y)

31709.3899514
32896.9476838
59934.2150057
40747.8110103
44218.9332442
31473.9630982
40569.551232
32387.5449939
26892.6793965
34427.4419038
24941.5885982
16208.4327959
20187.7253631
10108.6200766
7650.59368581
3529.39766403
1668.06150232
1298.85389873
1143.95624099
946.075127396
910.067708474
914.50385551
1003.38861006
977.286941983
979.075612881
975.659308254
897.877350518
947.380559676
968.664798122
944.379542539
882.633732209
782.480805034
655.564907246
530.144720192
460.131238263
504.780380465
692.727533937
700.512728896
638.730854217
557.819409368
442.794120512
479.322930003
490.864507657
427.954813594
331.51127237
342.588367041
439.207246912
501.623792999
537.098120012
630.690391072
702.264692319
688.088557288
684.870285146
703.110821032
736.577410761
802.689197556
878.85768503
1141.21686844
1419.98792186
1915.33279546
2047.36482356
2191.24562142
2281.11094893
2429.34558694
2613.90291748
3189.24752746
3792.61549606
5792.27751646
8017.63440625
14401.4571585
11076.2159709
8994.70215

0.398063794869
0.396872912089
0.395689636612
0.394513894489
0.393345612751
0.392184719394
0.391031143363
0.389884814532
0.388745663696
0.387613622546
0.386488623664
0.385370600502
0.384259487368
0.383155219416
0.382057732627
0.380966963798
0.379882850532
0.378805331218
0.377734345025
0.376669831883
0.375611732477
0.374559988233
0.373514541303
0.372475334557
0.371442311572
0.370415416617
0.369394594647
0.368379791288
0.367370952832
0.366368026218
0.365370959031
0.364379699488
0.363394196426
0.362414399297
0.361440258157
0.360471723655
0.359508747025
0.35855128008
0.357599275199
0.356652685321
0.355711463934
0.354775565072
0.353844943301
0.352919553715
0.351999351927
0.35108429406
0.350174336742
0.349269437098
0.34836955274
0.347474641764
0.34658466274
0.345699574707
0.344819337163
0.343943910065
0.343073253814
0.342207329256
0.34134609767
0.340489520767
0.33963756068
0.338790179959
0.337947341566
0.337109008869
0.336275145634
0.335445716024
0.334620684591
0.333800016268
0.332983676369
0

0.157157365592
0.15700421741
0.156851404551
0.156698925854
0.156546780159
0.156394966315
0.156243483175
0.156092329597
0.155941504446
0.155791006591
0.155640834904
0.155490988267
0.155341465564
0.155192265684
0.155043387523
0.15489482998
0.15474659196
0.154598672374
0.154451070136
0.154303784167
0.154156813391
0.154010156739
0.153863813144
0.153717781547
0.153572060892
0.153426650128
0.15328154821
0.153136754095
0.152992266748
0.152848085137
0.152704208234
0.152560635017
0.152417364469
0.152274395575
0.152131727327
0.151989358721
0.151847288757
0.15170551644
0.151564040779
0.151422860789
0.151281975486
0.151141383895
0.15100108504
0.150861077955
0.150721361674
0.150581935238
0.150442797689
0.150303948078
0.150165385456
0.150027108881
0.149889117413
0.149751410117
0.149613986063
0.149476844324
0.149339983978
0.149203404107
0.149067103795
0.148931082133
0.148795338214
0.148659871136
0.148524680001
0.148389763913
0.148255121983
0.148120753324
0.147986657053
0.147852832291
0.147719278163
0

0.103385024825
0.103324085871
0.103263224001
0.103202439061
0.103141730896
0.103081099353
0.103020544278
0.102960065519
0.102899662922
0.102839336336
0.10277908561
0.102718910591
0.102658811129
0.102598787074
0.102538838276
0.102478964584
0.102419165851
0.102359441927
0.102299792664
0.102240217913
0.102180717528
0.102121291362
0.102061939267
0.102002661097
0.101943456707
0.10188432595
0.101825268682
0.101766284758
0.101707374034
0.101648536366
0.10158977161
0.101531079623
0.101472460263
0.101413913386
0.101355438852
0.101297036518
0.101238706244
0.101180447887
0.101122261309
0.101064146369
0.101006102928
0.100948130845
0.100890229982
0.100832400201
0.100774641364
0.100716953332
0.100659335968
0.100601789135
0.100544312697
0.100486906517
0.100429570459
0.100372304388
0.100315108168
0.100257981665
0.100200924744
0.100143937271
0.100087019112
0.100030170134
0.0999733902032
0.0999166791877
0.0998600369549
0.0998034633728
0.0997469583097
0.0996905216343
0.0996341532158
0.0995778529235
0.099

0.0789598243277
0.0789253988013
0.0788910047313
0.0788566420732
0.0788223107824
0.0787880108147
0.0787537421258
0.0787195046713
0.0786852984072
0.0786511232895
0.0786169792742
0.0785828663175
0.0785487843755
0.0785147334047
0.0784807133613
0.0784467242018
0.0784127658829
0.0783788383612
0.0783449415934
0.0783110755363
0.0782772401468
0.078243435382
0.0782096611988
0.0781759175544
0.0781422044062
0.0781085217113
0.0780748694272
0.0780412475113
0.0780076559213
0.0779740946148
0.0779405635495
0.0779070626832
0.0778735919738
0.0778401513793
0.0778067408577
0.0777733603672
0.077740009866
0.0777066893124
0.0776733986648
0.0776401378816
0.0776069069213
0.0775737057426
0.0775405343042
0.0775073925648
0.0774742804833
0.0774411980187
0.0774081451299
0.077375121776
0.0773421279163
0.0773091635099
0.0772762285161
0.0772433228944
0.0772104466043
0.0771775996053
0.0771447818571
0.0771119933193
0.0770792339517
0.0770465037143
0.0770138025669
0.0769811304696
0.0769484873824
0.0769158732656
0.076883288

...

0.00523897556679
0.00523887120879
0.00523876685513
0.00523866250583
0.00523855816086
0.00523845382025
0.00523834948399
0.00523824515208
0.0052381408245
0.00523803650128
0.00523793218241
0.00523782786788
0.0052377235577
0.00523761925186
0.00523751495036
0.00523741065321
0.00523730636041
0.00523720207195
0.00523709778784
0.00523699350807
0.00523688923264
0.00523678496156
0.00523668069482
0.00523657643242
0.00523647217437
0.00523636792065
0.00523626367128
0.00523615942626
0.00523605518557
0.00523595094922
0.00523584671722
0.00523574248955
0.00523563826623
0.00523553404724
0.0052354298326
0.00523532562229
0.00523522141632
0.00523511721469
0.0052350130174
0.00523490882445
0.00523480463584
0.00523470045156
0.00523459627163
0.00523449209603
0.00523438792476
0.00523428375783
0.00523417959524
0.00523407543699
0.00523397128307
0.00523386713348
0.00523376298823
0.00523365884732
0.00523355471074
0.00523345057849
0.00523334645058
0.005233242327
0.00523313820776
0.00523303409284
0.00523292998226
0.0

0.00518651559031
0.00518641340524
0.00518631122437
0.0051862090477
0.00518610687525
0.005186004707
0.00518590254295
0.00518580038312
0.00518569822748
0.00518559607605
0.00518549392883
0.00518539178581
0.00518528964699
0.00518518751238
0.00518508538197
0.00518498325577
0.00518488113376
0.00518477901596
0.00518467690237
0.00518457479298
0.00518447268778
0.00518437058679
0.00518426849
0.00518416639741
0.00518406430902
0.00518396222484
0.00518386014485
0.00518375806906
0.00518365599747
0.00518355393009
0.0051834518669
0.00518334980792
0.00518324775312
0.00518314570253
0.00518304365614
0.00518294161394
0.00518283957594
0.00518273754214
0.00518263551254
0.00518253348713
0.00518243146592
0.00518232944891
0.00518222743609
0.00518212542746
0.00518202342304
0.0051819214228
0.00518181942677
0.00518171743492
0.00518161544728
0.00518151346382
0.00518141148456
0.00518130950949
0.00518120753861
0.00518110557194
0.00518100360945
0.00518090165116
0.00518079969705
0.00518069774714
0.00518059580142
0.005

0.00513624263698
0.00513614250956
0.00513604238621
0.00513594226694
0.00513584215174
0.00513574204061
0.00513564193356
0.00513554183057
0.00513544173167
0.00513534163683
0.00513524154607
0.00513514145937
0.00513504137675
0.0051349412982
0.00513484122372
0.00513474115332
0.00513464108698
0.00513454102472
0.00513444096652
0.00513434091239
0.00513424086233
0.00513414081635
0.00513404077443
0.00513394073658
0.00513384070279
0.00513374067308
0.00513364064743
0.00513354062585
0.00513344060834
0.0051333405949
0.00513324058552
0.00513314058021
0.00513304057896
0.00513294058179
0.00513284058868
0.00513274059963
0.00513264061465
0.00513254063373
0.00513244065688
0.0051323406841
0.00513224071537
0.00513214075071
0.00513204079012
0.00513194083358
0.00513184088112
0.00513174093271
0.00513164098837
0.00513154104809
0.00513144111187
0.00513134117972
0.00513124125162
0.00513114132759
0.00513104140761
0.0051309414917
0.00513084157985
0.00513074167206
0.00513064176833
0.00513054186866
0.00513044197305
0

0.00508599505724
0.00508589696243
0.00508579887157
0.00508570078465
0.00508560270167
0.00508550462264
0.00508540654755
0.00508530847641
0.0050852104092
0.00508511234595
0.00508501428663
0.00508491623126
0.00508481817983
0.00508472013234
0.00508462208879
0.00508452404919
0.00508442601352
0.0050843279818
0.00508422995402
0.00508413193017
0.00508403391027
0.00508393589431
0.00508383788229
0.00508373987421
0.00508364187006
0.00508354386986
0.00508344587359
0.00508334788126
0.00508324989287
0.00508315190842
0.00508305392791
0.00508295595133
0.0050828579787
0.00508276001
0.00508266204523
0.00508256408441
0.00508246612751
0.00508236817456
0.00508227022554
0.00508217228045
0.0050820743393
0.00508197640209
0.00508187846881
0.00508178053947
0.00508168261406
0.00508158469258
0.00508148677504
0.00508138886143
0.00508129095175
0.00508119304601
0.0050810951442
0.00508099724632
0.00508089935238
0.00508080146237
0.00508070357628
0.00508060569413
0.00508050781591
0.00508040994163
0.00508031207128
0.005

0.00503733849454
0.00503724234525
0.00503714619977
0.00503705005812
0.00503695392029
0.00503685778628
0.0050367616561
0.00503666552973
0.00503656940718
0.00503647328846
0.00503637717356
0.00503628106247
0.00503618495521
0.00503608885177
0.00503599275215
0.00503589665634
0.00503580056435
0.00503570447619
0.00503560839184
0.00503551231131
0.0050354162346
0.00503532016171
0.00503522409263
0.00503512802737
0.00503503196593
0.00503493590831
0.0050348398545
0.00503474380451
0.00503464775833
0.00503455171597
0.00503445567743
0.00503435964271
0.00503426361179
0.00503416758469
0.00503407156141
0.00503397554194
0.00503387952629
0.00503378351445
0.00503368750643
0.00503359150221
0.00503349550181
0.00503339950523
0.00503330351245
0.0050332075235
0.00503311153835
0.00503301555701
0.00503291957948
0.00503282360578
0.00503272763587
0.00503263166978
0.0050325357075
0.00503243974904
0.00503234379438
0.00503224784354
0.0050321518965
0.00503205595327
0.00503196001385
0.00503186407825
0.00503176814645
0.0

In [76]:
my_pred = clf.predict(X)
hard_pred = my_pred.argmax(axis=1)

In [78]:
np.allclose(y,hard_pred)

True

In [2]:
test = np.array([[1,2,3,4],[6,7,8,1],[-1,0,-1,0]])
test

array([[ 1,  2,  3,  4],
       [ 6,  7,  8,  1],
       [-1,  0, -1,  0]])

13.490737563232042

In [51]:
l = test.max(axis=1)
l

array([4, 8, 0])

In [53]:
lse = np.log(np.sum(np.exp(test - l[:,None]), axis = 1))

In [54]:
l + lse

array([ 4.4401897 ,  8.4082124 ,  1.00640887])